In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('./lib')
from Calibrator import Calibrator
from FX_params import FX_params
from FX_market_data import  FX_market_data
from Calibrated_params import Calibrated_params
from Strike_calculator import Strike_calcurator

#### input parameters here

In [ ]:
# input parameters here

# foreign rate
r_f = 3.46
#  domestic rate
r_d = 2.94
# term from spot date to Fixing date
T = 1.0

# ATM volatility
sigma_atm = 18.25
# butterfly 25 volatility
sigma_bf_25 = (0.950 - sigma_atm)
# risk riversal 25 volatility
sigma_rr_25 = -0.600 / 100
# butterfly 10 volatility
sigma_bf_10 = (3.806 - sigma_atm)
# risk riversal 10 volatility
sigma_rr_10 = -1.208

# spot rate
spot = 1.3465

# fly adjustment
fly_adjust_param = 0.

# initial SABR parameters
ini_alpha = 0.17431060
ini_nu = 0.81694072
ini_rho = -0.11268306
#initial smile strike
ini_K_25_d_p = 1.2034
ini_K_25_d_c = 1.5410
ini_K_10_d_p = 1.3
ini_K_10_d_c = 1.44

In [ ]:
r_f /= 100
r_d /= 100

sigma_atm /= 100
sigma_bf_25 /= 100
sigma_rr_25 /= 100
sigma_bf_10 /= 100
sigma_rr_10 /= 100

"""Df_f = 1 / math.exp(r_f * T)
Df_d = 1 / math.exp(r_d * T)"""
Df_f = 0.966001
Df_d = 0.971049

F_0 = spot * Df_f / Df_d
print(f"F_0 = {F_0}")

params = FX_params(r_d, r_f, T)
volas = FX_market_data(sigma_atm, sigma_bf_25, sigma_rr_25, sigma_bf_10, sigma_rr_10)
ini_calibrated_params = Calibrated_params(ini_alpha, ini_nu, ini_rho, ini_K_25_d_p, ini_K_25_d_c , ini_K_10_d_p, ini_K_10_d_c)

In [ ]:
strike_calcurator = Strike_calcurator(params, F_0, volas)

K_atm = strike_calcurator.get_atm_strike()
K_25_d_p_ms = strike_calcurator.get_ms_strike(-0.25)
K_25_d_c_ms = strike_calcurator.get_ms_strike(0.25)
K_10_d_p_ms = strike_calcurator.get_ms_strike(-0.10)
K_10_d_c_ms = strike_calcurator.get_ms_strike(0.10)
print(f"K_atm = {K_atm}")
print(f"K_25_d_p_ms = {K_25_d_p_ms}")
print(f"K_25_d_c_ms = {K_25_d_c_ms}")
print(f"K_10_d_p_ms = {K_10_d_p_ms}")
print(f"K_10_d_c_ms = {K_10_d_c_ms}")

In [ ]:
calibrator = Calibrator(params, F_0 , volas, fly_adjust_param)
model = calibrator.calibrate(ini_calibrated_params)
sabr_params = model.SABR_params()
print(f"alpha : {sabr_params['alpha']}, beta : {sabr_params['beta']}, nu : {sabr_params['nu']}, rho : {sabr_params['rho']}")

In [ ]:
strike_list = np.arange(0.9, 1.9, 0.01)
calc_price_vector = np.vectorize(model.calc_plain_vanilla_price)
price_list = calc_price_vector(strike_list, True)

fig, ax = plt.subplots(1, 1, figsize = (15, 9))
ax.plot(K_atm, volas.Sigma_atm(), 'o', label = "Market ATM")
ax.plot(K_25_d_p_ms, volas.Sigma_ms_25(), 'o', label = "Market 25% Strangle (put)")
ax.plot(K_25_d_c_ms, volas.Sigma_ms_25(), 'o', label = "Market 25% Strangle (call)")
ax.plot(K_10_d_p_ms, volas.Sigma_ms_10(), 'o', label = "Market 10% Strangle (put)")
ax.plot(K_10_d_c_ms, volas.Sigma_ms_10(), 'o', label = "Market 10% Strangle (call)")
ax.plot(strike_list, price_list, label = "smile")
ax.grid(True)
ax.legend()
pass